## Example for transferring ESM atm output to be used as OM2 atmospheric forcing
This notebook generates files for all variables listed in atmosphere/forcing.json except for river runoff and calving - see Eg_make_friver_variable.
Uses 3hourly output from SSP585 runs.

In [1]:
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
## SSP runs

In [3]:
cd /g/data/p73/archive/CMIP6/ACCESS-ESM1-5/SSP-585-10-re1/history/atm/netCDF

/g/data/p73/archive/CMIP6/ACCESS-ESM1-5/SSP-585-10-re1/history/atm/netCDF


In [4]:
ls

SSP-585-10-re1.pa-201501_mon.nc  SSP-585-10-re1.pi-201501_3h.nc
SSP-585-10-re1.pa-201502_mon.nc  SSP-585-10-re1.pi-201502_3h.nc
SSP-585-10-re1.pa-201503_mon.nc  SSP-585-10-re1.pi-201503_3h.nc
SSP-585-10-re1.pa-201504_mon.nc  SSP-585-10-re1.pi-201504_3h.nc
SSP-585-10-re1.pa-201505_mon.nc  SSP-585-10-re1.pi-201505_3h.nc
SSP-585-10-re1.pa-201506_mon.nc  SSP-585-10-re1.pi-201506_3h.nc
SSP-585-10-re1.pa-201507_mon.nc  SSP-585-10-re1.pi-201507_3h.nc
SSP-585-10-re1.pa-201508_mon.nc  SSP-585-10-re1.pi-201508_3h.nc
SSP-585-10-re1.pa-201509_mon.nc  SSP-585-10-re1.pi-201509_3h.nc
SSP-585-10-re1.pa-201510_mon.nc  SSP-585-10-re1.pi-201510_3h.nc
SSP-585-10-re1.pa-201511_mon.nc  SSP-585-10-re1.pi-201511_3h.nc
SSP-585-10-re1.pa-201512_mon.nc  SSP-585-10-re1.pi-201512_3h.nc
SSP-585-10-re1.pa-201601_mon.nc  SSP-585-10-re1.pi-201601_3h.nc
SSP-585-10-re1.pa-201602_mon.nc  SSP-585-10-re1.pi-201602_3h.nc
SSP-585-10-re1.pa-201603_mon.nc  SSP-585-10-re1.pi-201603_3h.nc
SSP-585-10-re1.pa-201604_mon.nc  SSP-585

In [5]:
from distributed import Client

client = Client(threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 14
Total threads: 14,Total memory: 63.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39099,Workers: 14
Dashboard: /proxy/8787/status,Total threads: 14
Started: Just now,Total memory: 63.00 GiB
Comm: tcp://127.0.0.1:38083,Total threads: 1
Dashboard: /proxy/33663/status,Memory: 4.50 GiB
Nanny: tcp://127.0.0.1:34877,


#### List of ESM variables needed for OM forcing
fld_s03i209
(time, lat, lon_u):10 METRE WIND U-COMP

fld_s03i210
(time, lat_v, lon):10 METRE WIND V-COMP

fld_s02i207
(time_0, lat, lon):
DOWNWARD LW RAD FLUX: SURFACE
units :W m-2

fld_s01i201
(time_0, lat, lon):
NET DOWN SURFACE SW FLUX: SW TS ONLY
units :W m-2

fld_s05i215
(time_0, lat, lon)
TOTAL SNOWFALL RATE: LS+CONV KG/M2/S

fld_s03i236
(time, lat, lon)
TEMPERATURE AT 1.5M
units :K

fld_s03i237
(time, lat, lon)
SPECIFIC HUMIDITY  AT 1.5M

fld_s05i214
(time_0, lat, lon)
TOTAL RAINFALL RATE: LS+CONV KG/M2/S

fld_s00i409
(time, lat, lon)
SURFACE PRESSURE AFTER TIMESTEPunits :Pa ## missing friver: total river runoff

In [6]:
var_list = ['fld_s03i209','fld_s03i210','fld_s02i207','fld_s01i201','fld_s05i215','fld_s03i236','fld_s03i237',
'fld_s05i214','fld_s00i409'] 

In [7]:
un_wanted_vars = ['fld_s08i223','fld_s08i223_0','fld_s08i229','fld_s08i230','fld_s03i316'] # definitely not needing soil

In [8]:
FileList = ['/g/data/p73/archive/CMIP6/ACCESS-ESM1-5/SSP-585-10-re1/history/atm/netCDF/SSP-585-10-re1.pi-2015' + '{0:02}'.format(i) + '_3h.nc' for i in range(1,13)]
FileList

['/g/data/p73/archive/CMIP6/ACCESS-ESM1-5/SSP-585-10-re1/history/atm/netCDF/SSP-585-10-re1.pi-201501_3h.nc',
 '/g/data/p73/archive/CMIP6/ACCESS-ESM1-5/SSP-585-10-re1/history/atm/netCDF/SSP-585-10-re1.pi-201502_3h.nc',
 '/g/data/p73/archive/CMIP6/ACCESS-ESM1-5/SSP-585-10-re1/history/atm/netCDF/SSP-585-10-re1.pi-201503_3h.nc',
 '/g/data/p73/archive/CMIP6/ACCESS-ESM1-5/SSP-585-10-re1/history/atm/netCDF/SSP-585-10-re1.pi-201504_3h.nc',
 '/g/data/p73/archive/CMIP6/ACCESS-ESM1-5/SSP-585-10-re1/history/atm/netCDF/SSP-585-10-re1.pi-201505_3h.nc',
 '/g/data/p73/archive/CMIP6/ACCESS-ESM1-5/SSP-585-10-re1/history/atm/netCDF/SSP-585-10-re1.pi-201506_3h.nc',
 '/g/data/p73/archive/CMIP6/ACCESS-ESM1-5/SSP-585-10-re1/history/atm/netCDF/SSP-585-10-re1.pi-201507_3h.nc',
 '/g/data/p73/archive/CMIP6/ACCESS-ESM1-5/SSP-585-10-re1/history/atm/netCDF/SSP-585-10-re1.pi-201508_3h.nc',
 '/g/data/p73/archive/CMIP6/ACCESS-ESM1-5/SSP-585-10-re1/history/atm/netCDF/SSP-585-10-re1.pi-201509_3h.nc',
 '/g/data/p73/archi

In [9]:
local_dir = '/g/data/k10/qo9901/ESMforcing_input/'

In [10]:
import netCDF4 as nc

In [11]:
import pandas as pd

# Example: create new daily time values for 2015
new_time = pd.date_range(start="2015-01-01", end="2015-12-31", freq="D")

## For 2015 - 2030

In [12]:
def make_one_year_forcing_ds(FileList):
    monthly_datasets = []
    
    #Load each month's dataset only once
    for mm in range(len(FileList)):
        print(f'Loading month {mm}')
        ds_month = xr.open_mfdataset(
            FileList[mm],
            parallel=True,
            combine='nested',
            drop_variables=un_wanted_vars,
            engine='netcdf4'
        ).load()
        monthly_datasets.append(ds_month)
    '''    
    # Normalize time coordinate
    if 'time_0' in ds_month.coords and 'time' not in ds_month.coords:
        ds_month = ds_month.rename({'time_0': 'time'})
    '''
    # Build variable for each array
    ds = []
    for var_name in var_list:
        print(f'Processing variable: {var_name}')
        var_store = []
        for ds_month in monthly_datasets:
            if var_name not in ds_month:
                raise ValueError(f"{var_name} not in dataset")
            var_store.append(ds_month[var_name])
        
        # Handle variable-specific time coordinate
        if 'time' in var_store[0].coords:
            ds_elem = xr.concat(var_store, dim='time')
        elif 'time_0' in var_store[0].coords:
            ds_elem = xr.concat(var_store, dim='time_0')
        else:
            raise ValueError(f"No known time coordinate for {var_name}")
        
        ds.append(ds_elem)
    one_yr_forcing_ds = xr.merge(ds)
    return one_yr_forcing_ds

In [13]:
def rename_calendar_yr(local_dir,yr,  name):
    # Open the file in append mode
    ds = nc.Dataset(local_dir + "ESM1p5_SSP585_full." + name + f".{yr:04}_{yr+1:04}.nc", "r+")
    time_var = ds.variables['time']
    
    # Update the calendar attribute
    time_var.calendar = 'gregorian'  # or 'gregorian', '360_day', etc.
    print(ds)
    # Close the dataset
    ds.close()

In [14]:
def saving_forcing_files(yr, local_dir, one_yr_forcing_ds):
    time_bnds_only = xr.open_mfdataset('/g/data/p73/archive/CMIP6/ACCESS-ESM1-5/SSP-585-10-re1/history/atm/netCDF/SSP-585-10-re1.pi-' + f'{yr:04}*_3h.nc',\
                                             parallel = True, drop_variables = un_wanted_vars,engine='netcdf4',**xarray_combine_by_coords_kwargs).time_0_bnds
    
    SSP_585_sel_t = xr.merge([one_yr_forcing_ds,time_bnds_only])
    SSP_585_sel_t
    # saving forcing variables that should be fed into om2 - see atmosphere/forcing.json file in exp dir

    small_ds = SSP_585_sel_t[["fld_s01i201", "time_0_bnds"]] 
    small_ds.rename({'time_0': 'time'}).to_netcdf(local_dir + "ESM1p5_SSP585_full.rsds." + f"{yr:04}_{yr+1:04}.nc")
    
    small_ds = SSP_585_sel_t[["fld_s02i207", "time_0_bnds"]] 
    small_ds.rename({'time_0': 'time'}).to_netcdf(local_dir + "ESM1p5_SSP585_full.rlds." + f"{yr:04}_{yr+1:04}.nc")
    
    small_ds = SSP_585_sel_t[["fld_s05i215", "time_0_bnds"]] 
    small_ds.rename({'time_0': 'time'}).to_netcdf(local_dir + "ESM1p5_SSP585_full.prsn." + f"{yr:04}_{yr+1:04}.nc")
    
    small_ds = SSP_585_sel_t[["fld_s05i214", "time_0_bnds"]] 
    small_ds.rename({'time_0': 'time'}).to_netcdf(local_dir + "ESM1p5_SSP585_full.prra." + f"{yr:04}_{yr+1:04}.nc")
    
    # assertion failed: get_var_dims: Unsupported dimension name lon_u 1 -error
    one_yr_forcing_ds.fld_s03i209.rename({'lon_u': 'lon'}).to_netcdf(local_dir + "ESM1p5_SSP585_full.uas_u." + f"{yr:04}_{yr+1:04}.nc")
    one_yr_forcing_ds.fld_s03i210.rename({'lat_v': 'lat'}).to_netcdf(local_dir + "ESM1p5_SSP585_full.vas_v." + f"{yr:04}_{yr+1:04}.nc")

    one_yr_forcing_ds.fld_s03i236.to_netcdf(local_dir + "ESM1p5_SSP585_full.tas." + f"{yr:04}_{yr+1:04}.nc")   
    one_yr_forcing_ds.fld_s03i237.to_netcdf(local_dir + "ESM1p5_SSP585_full.huss." + f"{yr:04}_{yr+1:04}.nc")
    one_yr_forcing_ds.fld_s00i409.to_netcdf(local_dir + "ESM1p5_SSP585_full.psl." + f"{yr:04}_{yr+1:04}.nc")
    
    rename_calendar_yr(local_dir, yr, 'uas_u')
    rename_calendar_yr(local_dir, yr,'vas_v')
    rename_calendar_yr(local_dir, yr, 'rsds')
    rename_calendar_yr(local_dir, yr, 'rlds')
    rename_calendar_yr(local_dir, yr, 'prsn')
    rename_calendar_yr(local_dir, yr, 'tas')
    rename_calendar_yr(local_dir, yr, 'huss')
    rename_calendar_yr(local_dir, yr,'psl')
    rename_calendar_yr(local_dir, yr, 'prra')

In [15]:
local_dir = '/g/data/k10/qo9901/ESMforcing_input/'

In [16]:
import netCDF4 as nc

In [18]:
xarray_combine_by_coords_kwargs=dict(
    compat="override",
    data_vars="minimal",
    coords="minimal", chunks = {'time':-1, 'time_0':-1}
)

In [19]:
%%time
master = []
for yr in range(2020, 2025):
    FileList = ['/g/data/p73/archive/CMIP6/ACCESS-ESM1-5/SSP-585-10-re1/history/atm/netCDF/SSP-585-10-re1.pi-'+ f'{yr:04}{i:02}_3h.nc'
        for i in range(1, 13)]
    print(yr)
    master.append(FileList)
    len(FileList)
    one_yr_forcing_ds = make_one_year_forcing_ds(FileList)
    saving_forcing_files(yr, local_dir, one_yr_forcing_ds)

2020
Loading month 0
Loading month 1
Loading month 2
Loading month 3
Loading month 4
Loading month 5
Loading month 6
Loading month 7
Loading month 8
Loading month 9
Loading month 10
Loading month 11
Processing variable: fld_s03i209
Processing variable: fld_s03i210
Processing variable: fld_s02i207
Processing variable: fld_s01i201
Processing variable: fld_s05i215
Processing variable: fld_s03i236
Processing variable: fld_s03i237
Processing variable: fld_s05i214
Processing variable: fld_s00i409
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    coordinates: height_0
    dimensions(sizes): time(2928), lat(145), lon(192)
    variables(dimensions): float64 time(time), float64 lat(lat), float64 lon(lon), float64 height(), float64 height_0(), float32 fld_s03i209(time, lat, lon)
    groups: 
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    coordinates: height_0
    dimensions(sizes): time(2928), lon(192), lat(144)
  

Now make sure that all data of the same year is in same nc file, because 3-hourly data has 1st Jan in previous year's file

In [59]:
var_name_list = ['rsds','rlds','prsn','prra','tas','huss','uas_u','vas_v', 'psl']#,'rsds','rlds''licalvf','friver','psl'

In [72]:
for var in var_name_list:
    print(var)
    test_combine = xr.open_mfdataset('/g/data/k10/qo9901/ESMforcing_input/ESM1p5_SSP585_full.' + var + '.*.nc', combine = 'by_coords')
    if 'time_0_bnds' in test_combine.data_vars:
        print('time_bnds rename')
        test_combine = test_combine.rename({'time_0_bnds':'time_bnds'})
    for yr in range(2015,2020):
        test_combine.sel(time = slice(str(yr), str(yr)))\
           .to_netcdf('/g/data/k10/qo9901/ESMforcing_input/ESM1p5_SSP585_full.' + var + '.' + str(yr) + '.nc') 

psl
